<a href="https://colab.research.google.com/github/eddy-udoh/Age-Calculator/blob/main/House_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. Load Data
data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# 2. Scale the data (Crucial for .h5 models)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Split into Train (80%) and Test (20%)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [15]:
# 1. Build the Neural Network
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(8,)), # 8 inputs for 8 house features
    tf.keras.layers.Dense(1) # 1 output for the Price
])

# 2. Compile and Train
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=10)

# 3. SAVE THE MODEL (Deliverable #1)
model.save('model.h5')
print("File 'model.h5' has been created!")

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


516/516 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 3.3966
Epoch 2/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.8118
Epoch 3/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6116
Epoch 4/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5155
Epoch 5/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4664
Epoch 6/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4361
Epoch 7/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4293
Epoch 8/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4267
Epoch 9/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4142
Epoch 10/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4061


File 'model.h5' has been created!


In [16]:
%%writefile app.py
from flask import Flask, render_template, request
import tensorflow as tf
import numpy as np

app = Flask(__name__)

# Load the brain (ensure you have already trained and saved model.h5)
try:
    model = tf.keras.models.load_model('model.h5')
except:
    print("Error: model.h5 not found. Please run the training cell first.")

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    # Gather inputs from the form
    input_data = [float(x) for x in request.form.values()]

    # Convert to array and predict
    final_input = np.array([input_data])
    prediction = model.predict(final_input)

    # Format the output (scaled value to actual dollar amount)
    output = round(float(prediction[0]), 2)

    return render_template('index.html',
                           prediction_text=f'Estimated House Value: ${output}0,000')

if __name__ == "__main__":
    app.run(debug=True)

Overwriting app.py


In [17]:
# Create the folder first
import os
if not os.path.exists('templates'):
    os.makedirs('templates')

# Create the HTML file
%%writefile templates/index.html
<!DOCTYPE html>
<html>
<head>
    <title>House Predictor</title>
    <style>
        body { font-family: Arial; margin: 40px; background-color: #f4f4f4; }
        input { display: block; margin-bottom: 10px; padding: 8px; width: 200px; }
    </style>
</head>
<body>
    <h1>House Price Prediction</h1>
    <p>Enter details below (e.g., Income, House Age, Rooms):</p>
    <form action="/predict" method="post">
        <input type="text" name="1" placeholder="MedInc (e.g. 3.5)" required>
        <input type="text" name="2" placeholder="House Age (e.g. 15)" required>
        <input type="text" name="3" placeholder="Avg Rooms (e.g. 5)" required>
        <input type="text" name="4" placeholder="Avg Bedrooms (e.g. 1)" required>
        <input type="text" name="5" placeholder="Population (e.g. 800)" required>
        <input type="text" name="6" placeholder="Avg Occupancy (e.g. 3)" required>
        <input type="text" name="7" placeholder="Latitude (e.g. 34.2)" required>
        <input type="text" name="8" placeholder="Longitude (e.g. -118.2)" required>
        <button type="submit">Predict Price</button>
    </form>
    <h2>{{ prediction_text }}</h2>
</body>
</html>

SyntaxError: invalid decimal literal (ipython-input-1940395587.py, line 13)

In [ ]:
import os
if not os.path.exists('templates'):
    os.makedirs('templates')

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>House Price Predictor</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 50px;
            background-color: #f0f2f5;
            display: flex;
            flex-direction: column;
            align-items: center;
        }
        .container {
            background: white;
            padding: 30px;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }
        input {
            display: block;
            margin-bottom: 15px;
            padding: 10px;
            width: 250px;
            border: 1px solid #ddd;
            border-radius: 5px;
        }
        button {
            width: 100%;
            padding: 10px;
            background-color: #007bff;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
        button:hover { background-color: #0056b3; }
        h1 { color: #333; }
        .result { color: #28a745; margin-top: 20px; font-weight: bold; }
    </style>
</head>
<body>
    <div class="container">
        <h1>House Price Prediction</h1>
        <form action="/predict" method="post">
            <input type="text" name="1" placeholder="Median Income (e.g. 3.5)" required>
            <input type="text" name="2" placeholder="House Age (e.g. 15)" required>
            <input type="text" name="3" placeholder="Average Rooms (e.g. 5)" required>
            <input type="text" name="4" placeholder="Average Bedrooms (e.g. 1)" required>
            <input type="text" name="5" placeholder="Population (e.g. 800)" required>
            <input type="text" name="6" placeholder="Average Occupancy (e.g. 3)" required>
            <input type="text" name="7" placeholder="Latitude (e.g. 34.2)" required>
            <input type="text" name="8" placeholder="Longitude (e.g. -118.2)" required>
            <button type="submit">Predict Price</button>
        </form>
        <div class="result">{{ prediction_text }}</div>
    </div>
</body>
</html>